In [1]:
import cv2
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt

# ---------- Config / Hyperparams ----------
DATA_FOLDER = "./Images/Data"   # ganti dengan folder berisi kandidat object
TARGET_PATH = "./Images/Object.png"     # ganti dengan path gambar target
SHOW_PLOTS = True

# Preprocessing choices: 'median' or 'gaussian'
PREPROCESS_MODES = ["median", "gaussian"]  # kita akan jalankan keduanya

# AKAZE params (menggunakan default)
AKAZE_THRESHOLD = None  # gunakan default AKAZE

# FLANN LSH params untuk descriptor biner (AKAZE menghasilkan binary-like descriptors)
FLANN_INDEX_LSH = 6
index_params = dict(algorithm = FLANN_INDEX_LSH,
                    table_number = 12, # 12..20 typical
                    key_size = 20,     # 12..20
                    multi_probe_level = 2)
search_params = dict(checks=50)

# Lowe ratio
LOWE_RATIO = 0.75

# Minimum good matches (after Lowe) to attempt homography
MIN_GOOD_MATCHES = 10

# RANSAC threshold (pixel)
RANSAC_REPROJ_THRESHOLD = 4.0

# ---------- Utilities ----------
def preprocess(img_gray, mode="median"):
    """Preprocess grayscale image: histogram equalization + blur (median or gaussian)."""
    # histogram equalization (works with single channel)
    img_eq = cv2.equalizeHist(img_gray)
    if mode == "median":
        # kernel size 5 as di-deskripsikan
        img_blur = cv2.medianBlur(img_eq, 5)
    elif mode == "gaussian":
        # kernel (3,3), sigmaX=0 as di-deskripsikan
        img_blur = cv2.GaussianBlur(img_eq, (3,3), 0)
    else:
        raise ValueError("Unknown preprocess mode")
    return img_blur

def detect_and_describe(img_gray, detector=None):
    """Return keypoints and descriptors using AKAZE."""
    if detector is None:
        detector = cv2.AKAZE_create()
    kps, desc = detector.detectAndCompute(img_gray, None)
    return kps, desc

def flann_match(desc1, desc2):
    """FLANN-based knnMatch (k=2) for binary descriptors using LSH index."""
    if desc1 is None or desc2 is None:
        return []
    # Ensure dtype is uint8 for binary descriptors (AKAZE often returns uint8)
    # OpenCV FLANN expects descriptors to be of type float32 for some algorithms; 
    # but for LSH it works with uint8. If errors occur, convert to np.uint8.
    if desc1.dtype != np.uint8:
        desc1 = desc1.astype(np.uint8)
    if desc2.dtype != np.uint8:
        desc2 = desc2.astype(np.uint8)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    # do knn match k=2
    matches = flann.knnMatch(desc1, desc2, k=2)
    return matches

def lowe_ratio_filter(knn_matches, ratio=0.75):
    """Apply Lowe's ratio test to knn matches (list of [m,n])"""
    good = []
    for m_n in knn_matches:
        if len(m_n) != 2:
            continue
        m, n = m_n
        if m.distance < ratio * n.distance:
            good.append(m)
    return good

def ransac_filter(kp1, kp2, good_matches, ransac_thresh=4.0):
    """Estimate homography and return inlier mask and inlier matches count."""
    if len(good_matches) < MIN_GOOD_MATCHES:
        return None, None, 0
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, ransac_thresh)
    if mask is None:
        return M, None, 0
    inliers = mask.ravel().tolist()
    inlier_count = int(np.sum(mask))
    return M, mask, inlier_count

def draw_matches(img1_color, img2_color, kp1, kp2, matches, mask=None, max_display=50, title=None):
    """Draw matched keypoints, optionally mask (inliers)."""
    draw_params = dict(matchColor = (0,255,0),
                       singlePointColor = None,
                       matchesMask = None if mask is None else mask.ravel().tolist(),
                       flags = 2)
    img_matches = cv2.drawMatches(img1_color, kp1, img2_color, kp2, matches[:max_display], None, **draw_params)
    if title:
        plt.figure(figsize=(12,6)); plt.title(title)
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# ---------- Main pipeline ----------
def evaluate_folder(target_path, folder_path, preprocess_mode="median", show_each=True):
    # load target
    tgt_color = cv2.imread(target_path)
    if tgt_color is None:
        raise FileNotFoundError(f"Target image not found: {target_path}")
    tgt_gray = cv2.cvtColor(tgt_color, cv2.COLOR_BGR2GRAY)
    tgt_prep = preprocess(tgt_gray, mode=preprocess_mode)
    kp_tgt, desc_tgt = detect_and_describe(tgt_prep)
    print(f"[{preprocess_mode}] Target: {len(kp_tgt)} keypoints, desc shape: {None if desc_tgt is None else desc_tgt.shape}")

    results = []
    # iterate candidates
    image_files = sorted(glob(os.path.join(folder_path, "*.*")))
    for img_path in image_files:
        img_color = cv2.imread(img_path)
        if img_color is None:
            continue
        img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        img_prep = preprocess(img_gray, mode=preprocess_mode)
        kp_img, desc_img = detect_and_describe(img_prep)
        # FLANN knn
        knn = flann_match(desc_tgt, desc_img)
        good = lowe_ratio_filter(knn, ratio=LOWE_RATIO)
        M, mask, inlier_count = (None, None, 0)
        if len(good) >= MIN_GOOD_MATCHES:
            M, mask, inlier_count = ransac_filter(kp_tgt, kp_img, good, ransac_thresh=RANSAC_REPROJ_THRESHOLD)
        # store metrics
        metrics = {
            "path": img_path,
            "num_kp": len(kp_img),
            "desc_shape": None if desc_img is None else desc_img.shape,
            "knn_total_pairs": len(knn),
            "good_matches_after_lowe": len(good),
            "inlier_count_ransac": inlier_count,
            "kp_tgt": len(kp_tgt)
        }
        results.append((metrics, img_color, kp_img, good, mask))
        if show_each:
            print(f"Candidate: {os.path.basename(img_path)} | kp:{metrics['num_kp']} | good:{metrics['good_matches_after_lowe']} | inliers:{metrics['inlier_count_ransac']}")
    # choose best by inlier_count (primary), then by good_matches
    results_sorted = sorted(results, key=lambda x: (x[0]['inlier_count_ransac'], x[0]['good_matches_after_lowe']), reverse=True)
    return results_sorted, tgt_color, kp_tgt, tgt_prep

# ---------- Run ----------
if __name__ == "__main__":
    all_mode_results = {}
    for mode in PREPROCESS_MODES:
        print("=== Running mode:", mode)
        res_sorted, tgt_color, kp_tgt, tgt_prep = evaluate_folder(TARGET_PATH, DATA_FOLDER, preprocess_mode=mode, show_each=True)
        # print top 3
        print("--- Top candidates for mode:", mode)
        for i, (metrics, img_color, kp_img, good, mask) in enumerate(res_sorted[:3]):
            print(f"Rank {i+1}: {os.path.basename(metrics['path'])} | kp:{metrics['num_kp']} | good:{metrics['good_matches_after_lowe']} | inliers:{metrics['inlier_count_ransac']}")
            if SHOW_PLOTS:
                title = f"Mode={mode} | Rank {i+1}: {os.path.basename(metrics['path'])} (good={metrics['good_matches_after_lowe']}, inliers={metrics['inlier_count_ransac']})"
                draw_matches(tgt_color, img_color, kp_tgt, kp_img, good, mask=mask, title=title)
        all_mode_results[mode] = res_sorted

    # Summary comparison between modes for top-1
    print("\n=== Summary comparison (top-1 each mode) ===")
    for mode, res_sorted in all_mode_results.items():
        if len(res_sorted) == 0:
            print(mode, "no candidates found")
            continue
        metrics = res_sorted[0][0]
        print(mode, "->", os.path.basename(metrics['path']), "| good:", metrics['good_matches_after_lowe'], "| inliers:", metrics['inlier_count_ransac'])


=== Running mode: median
[median] Target: 1788 keypoints, desc shape: (1788, 61)
Candidate: Charmender.png | kp:56 | good:81 | inliers:18
Candidate: Gastly.png | kp:942 | good:41 | inliers:8
Candidate: Piplup.png | kp:230 | good:51 | inliers:14
Candidate: Quaxly.png | kp:149 | good:72 | inliers:17
Candidate: Rowlet.png | kp:1608 | good:26 | inliers:5
Candidate: Squirtle.png | kp:1363 | good:26 | inliers:0
Candidate: Treecko.png | kp:281 | good:91 | inliers:12
--- Top candidates for mode: median
Rank 1: Charmender.png | kp:56 | good:81 | inliers:18


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\features2d\src\draw.cpp:228: error: (-201:Incorrect size of input array) matchesMask must have the same size as matches1to2 in function 'cv::drawMatches'
